# Grocery List

In [1]:
import requests
import pandas as pd
import json
from collections import defaultdict

# --- Configuration ---
HASURA_ENDPOINT = "http://localhost:8080/v1/graphql"  # Replace with your actual Hasura endpoint
HASURA_HEADERS = {
    "Content-Type": "application/json",
    "x-hasura-admin-secret": "your-secret"  # Replace with your Hasura admin secret if needed
}
SOURCE_NAME = "default"  # Change if your Hasura source has another name

QUERY_FILE = "queries/grocery_list.gql"
VARIABLE_FILE = "queries/grocery_list_input.json"
OUTPUT_FILE = "queries/grocery_list_output.json"
OUTPUT_FILE_CSV = "data/grocery_list/grocery_list_output.csv"

## Call API

In [2]:
# Read query
with open(QUERY_FILE, "r") as gql_file:
    query = gql_file.read()

# Read variables
with open(VARIABLE_FILE, "r") as var_file:
    variables = json.load(var_file)

# === Send GraphQL request ===
response = requests.post(
    HASURA_ENDPOINT,
    headers=HASURA_HEADERS,
    json={
        "query": query,
        "variables": variables
    }
)

# === Handle response ===
if response.status_code == 200:
    data = response.json()
    with open(OUTPUT_FILE, "w") as f:
        json.dump(data, f, indent=2)
    print(f"✅ Response saved to {OUTPUT_FILE}")
else:
    print(f"❌ Request failed: {response.status_code} - {response.text}")

✅ Response saved to queries/grocery_list_output.json


## Shape API Response

In [24]:
# Load your JSON (replace with actual loading logic)
with open(OUTPUT_FILE) as f:
    data = json.load(f)

# Grouped result: (store_section_id, ingredient_id, display) -> total_qty
grouped = defaultdict(float)

for item in data["data"]["map_recipe_ingredient"]:
    section = item["ingredient_by_ingredient_id"]["store_section_id"]
    ingredient = item["ingredient_id"]
    display = item["unit_by_unit_id"]["display"]
    qty = float(item["ingredient_qty"])
    key = (section, ingredient, display)
    grouped[key] += qty

# Format results
result = [
    {
        "store_section_id": k[0],
        "ingredient_id": k[1],
        "unit_display": k[2],
        "total_qty": v
    }
    for k, v in grouped.items()
]

# Pretty-print
# import pprint
# pprint.pprint(result)


## Save as .csv

In [25]:
# Convert to DataFrame
df = pd.DataFrame(
    [
        {
            "store_section_id": k[0],
            "ingredient_id": k[1],
            "unit_display": k[2],
            "total_qty": v
        }
        for k, v in grouped.items()
    ]
)

# Save to CSV
df.to_csv(OUTPUT_FILE_CSV, index=False)

print("CSV saved as grouped_ingredients.csv")

CSV saved as grouped_ingredients.csv
